In [50]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

import bs4
import os, re, requests, time
from typing import List, Tuple, Dict


gid = os.getenv('GENIUS_ID')
gsec = os.getenv('GENIUS_SECRET')
gtok = os.getenv('GENIUS_TOKEN')


class Lyrics(object):


    def __init__(self):
        self.gen_url = 'https://genius.com/'
        self.list_url = 'https://www.song-list.net/'
        

    def just_replace_strings_with_dashes(self, artist:str) -> str:
        """
        replaces ' ' with '-', that's it
        """
        data = re.sub(' ', '-', artist)

        return data
    
    
    def just_replace_strings_with_nothing(self, artist:str) -> str:
        """
        replaces ' ' with '', that's it
        """
        data = re.sub(' ', '', artist)
        
        return data
    
        
    def get_song_list(self, artist:str) -> List[str]:
        """
        Gets discography of artist for lyric grabbing
        I'm limited by the technology of my time (my current regex understanding)
        So this is pretty pretty pretty inneficient but reaching marginal returns
        Regarding regex iteration
        """
        artist = self.just_replace_strings_with_nothing(artist)
        
        url = self.list_url + artist + '/songs'
        
        resp = requests.get(url)

        content = bs4.BeautifulSoup(resp.content)

        song_list = content.text[content.text.index('MP3s')+5:content.text.index('About Song List')]

        song_list = re.sub('\n', ',', song_list)
        song_list = re.sub(',+', ',', song_list)
        song_list = re.sub(', ,', ', ', song_list)

        song_list = re.split(',', song_list)
        for i in range(len(song_list)):
            song_list[i] = song_list[i].lstrip(' ')
            song_list[i] = re.sub("[.,']", '', song_list[i])
            song_list[i] = re.sub("&", 'and', song_list[i])
            song_list[i] = re.sub('\s+', ' ', song_list[i])

        song_list = [i for i in song_list if i != '']

        return song_list
    
    
    def clean_lyrics_response(self, lyrics: str) -> str:
        """
        Cleans lyric response
        """
        lyrics = re.sub("'", '', lyrics)
        lyrics = re.sub('\W', ' ', lyrics)
        needs_space = re.finditer(r'([a-z][A-Z])', lyrics)

        for i in needs_space:
            group = i.group()[-2:]
            lyrics = re.sub(group, '{} {}'.format(group[0], group[1]), lyrics)

        lyrics = re.split('(Verse|Chorus|Outro)', lyrics)
        keep = []
        for i in range(len(lyrics)):
            if lyrics[i] != ' ':
                keep.append(lyrics[i])

        lyrics = keep
        del keep

        titles = lyrics[::2]
        words = lyrics[1::2]
        for i in range(len(titles)):
            words[i] = titles[i] + words[i]

        lyrics = "".join(i+' \n ' for i in words)

        return lyrics
    

    def get_genius_page(self, artist:str, song:str) -> str:
        """
        gets page of lyrics from genius
        """

        artist = self.just_replace_strings_with_dashes(artist)
        song = self.just_replace_strings_with_dashes(song)

        url = self.gen_url + artist + '-' + song + '-lyrics'

        resp = requests.get(url)
        
        if resp.status_code == 200:
            try:
                content = bs4.BeautifulSoup(resp.content)
                lyrics = content.text[content.text.rindex('[Verse 1]'):content.text.index('Embed')]
                lyrics = self.clean_lyrics_response(lyrics)
                return lyrics
            
            except (ValueError, IndexError) as e:
                print('Lyrics not found {}, due to error {}'.format(song, e))
                
            try:
                lyrics = content.text[content.text.rindex('[Verse]'):content.text.index('Embed')]
                lyrics = self.clean_lyrics_response(lyrics)
                return lyrics
            
            except ValueError as e:
                print('Lyrics not found {}, due to error {}, single verse song'.format(song, e))


        
    def get_all_artists_lyrics(self, artist:str) -> List[Dict]:
        """
        Gets all songs lyrics for a given artist
        artist input should look like "queens of the stone age"
        """
        artist = artist.lower()
        song_list = self.get_song_list(artist)
        

        
        lyric_dict = {}
        for i in song_list:
            lyrics = self.get_genius_page(artist, i)
            lyric_dict[i] = lyrics

        return lyric_dict
            

In [28]:
L = Lyrics()

resp = L.get_song_list('queens of the stone age')


In [51]:
L = Lyrics()

resp = requests.get('https://genius.com/queens-of-the-stone-age-this-lullaby-lyrics')


In [52]:
L = Lyrics()

#lyrics = L.get_genius_page('queens of the stone age', 'make it wit chu')
resp = L.get_all_artists_lyrics('queens of the stone age')

resp

['Like Clockwork', '3s and 7s', 'A Song For The Dead', 'A Song For The Deaf', 'Another Love Song', 'Auto Pilot', 'Avon', 'Battery Acid', 'Better Living Through Chemistry', 'Broken Box', 'Burn The Witch', 'Do It Again', 'Domesticated Animals', 'Everybody Knows That Youre Insane', 'Fairweather Friends', 'Feel Good Hit of the Summer', 'Feet Dont Fail Me', 'First It Giveth', 'Fortress', 'Give The Mule What He Wants', 'Go With The Flow', 'God Is In The Radio', 'Gonna Leave You', 'Hangin Tree', 'Head Like A Haunted House', 'Hideaway', 'Hispanic Impressions', 'How To Handle A Rope', 'I Appear Missing', 'I Never Came', 'I Sat By The Ocean', 'I Think I Lost My Headache', 'I Was A Teenage Hand Model', 'Im Designer', 'If I Had A Tail', 'If Only', 'In My Head', 'In the Fade', 'Into the Hollow', 'Kalopsia', 'Keep Your Eyes Peeled', 'Leg of Lamb', 'Lightning Song', 'Little Sister', 'Long Slow Goodbye', 'Make It Wit Chu', 'Medication', 'Mexicola', 'Misfit Love', 'Monsters in the Parasol', 'Mosquito S

Verse 1 Holding hands Skipping like a stone On our way To see what we have done The first to speak Is the first to lie The children cross Their hearts and hope to die  
 Chorus 1 Bite your tongue Swear to keep your mouth shut  
 Verse 2 Ask yourself Will I burn in Hell Then write it down And cast it in the well There they are The mob it cries for blood To twist the tale Into fire wood Fan the flames With a little lie Then turn your cheek Until the fire dies The skin it peels Like the truth  away What it was I will never say  
 Chorus 2 Bite your tongue Swear to keep your mouth shut Make up something Make up something good  
 Outro Holding hands Skipping like a stone Burn the witch Burn to ash and bone Burn the witch Burn to ash and bone Burn the witch Burn to ash and bone5 
 
Verse 1 Hey  Hey  Hey I fall over and over and over Over  over on you I get ill  I get ill  I get ill Youre the only one Im into  
 Chorus 1 You and me fit so tight  
 Verse 2 I go lower and lower and lower  lower

Verse 1 I thought I saw him on the video A trance is keepin him under I know that God is in the radio Ch ch ch checking the station The narcotic  from me to you Just a call in the medium I know you hear it  I hear it too Its everywhere that I go  
 Chorus You come back another day And do no wrong You come back another day And do no wrong You come back another day And do no wrong You come back another day And do no wrong  
 Verse 2 They say the devil is paranoid Always tryin to cover But God is leaking through the stereo Between the station to station You believe it  I know you do You wont admit it or say so I know that God is in the radio Just repeatin a slogan  
 Chorus You come back another day And do no wrong You come back another day And do no wrong You come back another day And do no wrong You come back another day And do no wrong Guitar Solo  Instrumental  
 Outro 4 
 
Verse 1 Wait by the phone Late all alone He cant help you But I know I could Pre  
 Chorus Three years too long 

Verse 1 I sat by the ocean And drank a potion  baby  to erase you Face down in the boulevard Yet  I couldnt face you There aint no use in crying It doesnt change anything So  baby  what good does it do Your friends  they all sympathize Maybe I don t need them to  
 Chorus Imagined Id be your one and only Instead  Im the lonely one You  me and a lie Silence is closer  we re passing ships in the night  
 Verse 2 Theres nowhere to run away Said   Boy  if you want love Youll have to go and find it with someone new Do you know who you really are Are you sure its really you  Lies are a funny thing They slip through your fingertips Because they never happened to you Time wounds all the heals as we fade out of view  
 Chorus Imagined Id be your one and only Instead Im the lonely one You  me and a lie Silence is closer  we re passing ships in the night You  me and a lie Silence is closer  we re passing ships in the night Into the light  left with nowhere to hide  
 Outro Closer and closer  we r

Lyrics not found Lightning-Song, due to error substring not found
Lyrics not found Lightning-Song, due to error substring not found, single verse song
Verse 1 Hey  sister  why you all alone Im standing out your window Hey  little sister  can I come inside  dear I want to show you all my love I want to be the only one I know you like nobody ever  baby  
 Chorus 1 Little sister  cant you find another way No more living life behind a shadow Little sister  cant you find another way No more living life behind a shadow  
 Verse 2 You whisper secrets in my ear Slowly dancing cheek to cheek Such a sweet thing when you open up  baby They say Ill only do you wrong Come together cause I understand Just who you really are  yeah  baby  
 Chorus 2 Little sister  cant you find another way No more living life behind a shadow Little sister  cant you find another way No more living life behind a shadow A shadow13 
 
Verse 1 Where have you gone again my sweet Everybody wants to know Where have you gone a

Verse 1 A little girl  walking down the street Godverdomme  you gotta be with me I know  you know Everybody  everyone  lets go  
 Chorus Oh baby  baby  youre so sweet Ah  Ah  woo   
 Verse 2 Het leven is hard  de mensen worden bang Im so old  youre so young I know  you know Everybody  everyone  lets go  
 Chorus Oh baby  baby  youre so sweet Goddamn  Ah  Ah  woo  Bridge Yeah  yeah  yeah  yeah Yeah  yeah  yeah  yeah   
 Verse 3 Ik denk van ja  maar nee Little girl  listen to what I say You know  I know Everybody  everyone  lets go  
 Chorus Oh baby  baby  youre so sweet Oh baby  baby  youre so sweet Goddamn  Ah  Ah  woo  
 
Verse 1 Who are you girl Who are you boy Bet I know what youre up to Can I come along Your home numbers on the wall I just had to call  had to Im not the only one Who will run with a knife  
 Chorus 1 Open up your eyes Open up your room Open up your arms  
 Verse 2 Eight six two seven eight Two six three seven eight nine Wheres your daddy now I heard he ran away Who 

Verse 1 I slipped Didnt mean  didnt mean to do it that way But I blew in on a whim  gone tomorrow   Shit Im gone today Bridge 1 Oh yeah  oh yeah Oh oh oh yeah Yeah  yeah  yeah  Yeah  yeah  yeah  Ooo ooo  woah  Yeah  yeah  yeah   
 Verse 2 Come  lets play along and let each other lose A win would cause an alarm Dont matter to me  dont matter to you Bridge Oh yeah  oh yeah Oh oh oh yeah  
 Chorus 1 I could keep you all for myself I know you gotta be free So free yourself I could keep you all for myself I know you gotta be free So free yourself Post  
 Chorus  Yeah  yeah  yeah  Ooo ooo  woah  Yeah  yeah  yeah   
 Verse 3 Self inflicted wound Your gift  impeccable aim Can really clear a room All the bodies piled up in your wake Bridge 2 Oh yeah It hurts  So bad It must  Oh yeah  
 Chorus 2 I could keep you all for myself I know you gotta be free So free yourself I could keep you all for myself I know you gotta be free So kill yourself Guitar Solo 3 
 
Lyrics not found Tension-Head, due to 

KeyboardInterrupt: 

In [6]:
len(resp)

344

In [402]:
lyrics

'Verse 1 You wanna know if I know why I cant say that I do Dont understand the evil eye Or how one becomes two And I just cant recall what started it all Or how to begin in the end I aint here to break it Just see how far it will bend Again and again  again and again  \n Chorus I wanna make it I wanna make it wit chu Anytime  anywhere I wanna make it Again and again I wanna make it wit chu  \n Verse 2 Sometimes the same is different But mostly its the same These mysteries of life That jus aint my thing If I told you that I knew about the sun and the moon Id be untrue The only thing I know for sure Is what I wanna do Anytime  anywhere  and I say babe  \n Chorus I wanna make it Again and again I wanna make it wit chu Anytime  anywhere I wanna make it Again and again I wanna make it wit chu I wanna make it Anytime  anywhere I wanna make it wit chu Yeah I wanna make it Whooo I wanna make it wit chu Guitar solo   \n Chorus Reprise I wanna make it Again and again I wanna make it wit chu I wa